<a href="https://colab.research.google.com/github/caplove/DX_LG_KOREA/blob/main/20_01_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <br>[ LG전자_DX_Intensive_Course  ] 자연어 처리 1<br><br> : Topic Modeling - Latent Dirichlet Allocation<br>

In [ ]:
# github에서 데이터 불러오기
!git clone https://github.com/KU-DIC/LG_natural_language_processing_day20

fatal: destination path 'LG_natural_language_processing_day20' already exists and is not an empty directory.


In [ ]:
! pip install pyLDAvis   # LDA Visualize tool 

     |████████████████████████████████| 1.7 MB 28.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f25c510bce1e19ff2d0a551ac5b2e32050ba963a103a4341167f348f62b2155d
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
# local에서 
! pip install numpy --upgrade

In [ ]:
! pip install pandas --upgrade

In [ ]:
! pip install matplotlib --upgrade

     |████████████████████████████████| 11.2 MB 29.7 MB/s 
     |████████████████████████████████| 899 kB 57.8 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# 모듈 불러오기
import random
import numpy as np
import pandas as pd
from collections import Counter    # list가 주어졌을때, 요소를 세는 모듈

import gensim   # word2vec 패키지  -- 안에 LDA tool 있음.
import nltk     # 영어단어 모음 툴?
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess  # 전처리
from gensim.parsing.preprocessing import STOPWORDS  # 전처리
from gensim.models import LdaModel   # 모델
from nltk.stem import WordNetLemmatizer, SnowballStemmer   # 전처리

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# <br>__1. Data: IMDB Dataset__
- 데이터 description
    - IMDB Dataset은 영화 사이트 IMDB의 리뷰 데이터로 구성된 데이터셋이다. 해당 데이터를 활용한 토픽 모델링 task는 텍스트 데이터를 input으로 받아 input을 관통하는 주요 주제와 해당 주제에 빈번하게 사용되는 단어들을 찾아내는 것을 목표로 한다. <br><br>

- 데이터 출처
    - https://www.imdb.com/interfaces/
    - https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
# 데이터 불러오기
# 각 리뷰를 하나의 문서로 고려하여 토픽 모델링을 진행함
corpus_path = '/content/LG_natural_language_processing_day20/input/imdb-data/IMDB_Dataset.csv'
documents = pd.read_csv(corpus_path, encoding='utf-8')   # 한글은 다른 encoding 옵션 사용해야 함
documents.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# 방식 2가지

# Stemming :  love   --> love
#             loving  --> love 로 통일하여, 단어수가 대폭적으로 준다.

# 문맥/맥락이 중요한 task는 Lenmatization 방식을 쓴다.
# Lenmatization  : 단어를 통일하되, 품사도 유지, 명사 --> 명사, 동사 --> 동사로 유지함.
# 한글은 형태소 분석기를 통해서 유사하게 작동한다. 
# KoNLPy 한국어 NLP 형태소 분석기


# Lemmatization 함수
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text))   # 2개를 다 사용함... 그 때 순서는 Lemnatization --> Stemming 순으로...

In [ ]:
stemmer.stem('living')
# 'live'

'live'

In [ ]:
lemmatizer=WordNetLemmatizer()
lemmatizer.lemmatize('living')
# 'living'

'living'

In [ ]:
# 둘다 적용
stemmer.stem(lemmatizer.lemmatize('living'))

'live'

In [ ]:
# KoNLP? 

# Tokenization & stopword 제거 & stemming & lemmatization

# 공백 --> 분리,  대문자 --> 소문자,  전치사,관사 (STOPWORDS) -->제거
# 여기서 text는 문장
def preprocess(text):
    result = []  # 빈 list
    for token in gensim.utils.simple_preprocess(text):    # 문장을 token으로 나누고, 대문자--> 소문자로 바꿈, token은 각각의 단어임
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:  # token이 stopwords 가 아니고 4글자 이상이면...
            result.append(lemmatize_stemming(token))      # result에 append
    return result

In [ ]:
# gensim.utils.simple_preprocess(documents['revies'][0][4])

In [ ]:
processed_docs = documents['review'].map(preprocess)   # .map 은 각각의 문장에 preprocess 수행하라 
processed_docs[:10]

0    [review, mention, watch, episod, hook, right, ...
1    [wonder, littl, product, film, techniqu, unass...
2    [thought, wonder, spend, time, summer, weekend...
3    [basic, famili, littl, jake, think, zombi, clo...
4    [petter, mattei, love, time, money, visual, st...
5    [probabl, time, favorit, movi, stori, selfless...
6    [sure, like, resurrect, date, seahunt, seri, t...
7    [amaz, fresh, innov, idea, air, year, brillian...
8    [encourag, posit, comment, film, look, forward...
9    [like, origin, wrench, laughter, like, movi, y...
Name: review, dtype: object

In [ ]:
# 문장들  --> 단어(token) 조합
# token --> index(숫자)

# Dictionary 학습
# Dictionary는 단어를 int 형식의 idx로 변환하는 encoder인 token2id를 학습함
dictionary = gensim.corpora.Dictionary(processed_docs)
print('dictionary size : %d' % len(dictionary))  # dictionary size : 65896   (유니크 단어의 숫자)

dictionary size : 65896


In [ ]:
# 빈도수를 기준으로 keep_n의 단어만 사용
dictionary.filter_extremes(no_below=50, no_above=0.1, keep_n=5000)   # 최소의 document(문장)의 수치 50 , 10% 아래의 문장개수에 등장, 상우 5000개 
print('dictionary size : %d' % len(dictionary))  # dictionary size : 5000

dictionary size : 5000


In [ ]:
# LDA의 학습 데이터 형식: list of list of (int, int) = (단어 index, 빈도수)
# Dictionary의 doc2bow 함수를 사용하여 단어의 list를 (단어, 빈도수)의 list로 변환함
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]   #  doc2bow 하면 (인덱스, 빈도) 로 mapping 한다.
for i, doc in enumerate(corpus):
    if i >= 5: break   # 5개 문장만 봄
    print(doc)  

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 4), (80, 2), (81, 1)]
[(13, 1), (34, 1), (37, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 2), (101, 2), (102, 2), (103, 2), (104, 1), (105, 1), (106, 1), (107, 1), (

---

# <br>__2. Model: Latent Dirichlet Allocation__

- LDA 설명 - **gensim.models.LdaModel()**
    - corpus: 문서 corpus
    - id2word: dictionary
    - num_topics: topic 개수<br><br>

In [ ]:
# LDA 모델 구축
# LDA 모델은 (topic, term) 행렬 정보와 topic probability 정보를 가지고 있음
num_topics = 10      # 모수
lda_model = LdaModel(corpus, id2word=dictionary, num_topics=num_topics, random_state=1234)

In [ ]:
# 토픽별 키워드 확인: get_topic_terms() 사용
for i in range(num_topics):
    keywords_idx = lda_model.get_topic_terms(i, 10)   # (몇번째 Topic, 몇개 Keyword)      (272,00082) (175,0.00806), ...

    keywords = []
    for idx, prob in keywords_idx:
        keywords.append(dictionary[idx])     #  단어형태로 바꿔서 keywords 배열에 append함.

    print('Topic: {} \nKeywords: {}'.format(i + 1, keywords)) # 확률값이 높은 상위 10개 idx를 찾고 이를 키워드로 단어반환함.

Topic: 1 
Keywords: ['horror', 'killer', 'blood', 'hous', 'dead', 'zombi', 'gore', 'night', 'flick', 'scari']
Topic: 2 
Keywords: ['school', 'kid', 'child', 'famili', 'home', 'wife', 'jack', 'mari', 'christma', 'help']
Topic: 3 
Keywords: ['game', 'space', 'human', 'team', 'earth', 'guy', 'alien', 'fight', 'special', 'monster']
Topic: 4 
Keywords: ['famili', 'father', 'mother', 'wife', 'child', 'relationship', 'live', 'husband', 'emot', 'daughter']
Topic: 5 
Keywords: ['murder', 'mysteri', 'style', 'lead', 'thriller', 'viewer', 'crime', 'audienc', 'shot', 'element']
Topic: 6 
Keywords: ['seri', 'book', 'anim', 'episod', 'version', 'laugh', 'read', 'joke', 'cartoon', 'voic']
Topic: 7 
Keywords: ['worst', 'wast', 'bore', 'poor', 'terribl', 'review', 'aw', 'money', 'hour', 'mayb']
Topic: 8 
Keywords: ['american', 'documentari', 'countri', 'polit', 'french', 'histori', 'differ', 'john', 'citi', 'live']
Topic: 9 
Keywords: ['beauti', 'excel', 'perfect', 'song', 'danc', 'episod', 'screen', '

In [ ]:
keywords_idx

[(272, 0.008498382),
 (175, 0.008067945),
 (300, 0.0073948624),
 (459, 0.006714058),
 (1604, 0.006499561),
 (1426, 0.005469366),
 (1112, 0.0046627056),
 (279, 0.004618133),
 (431, 0.0045944797),
 (321, 0.0034566047)]

In [ ]:
# 토픽별 단어의 분포 확인: print_topics() 사용
for i, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(i + 1, topic))

Topic: 1 
Words: 0.025*"horror" + 0.008*"killer" + 0.007*"blood" + 0.007*"hous" + 0.007*"dead" + 0.006*"zombi" + 0.006*"gore" + 0.006*"night" + 0.005*"flick" + 0.005*"scari"
Topic: 2 
Words: 0.012*"school" + 0.009*"kid" + 0.009*"child" + 0.007*"famili" + 0.004*"home" + 0.004*"wife" + 0.004*"jack" + 0.004*"mari" + 0.004*"christma" + 0.004*"help"
Topic: 3 
Words: 0.011*"game" + 0.007*"space" + 0.005*"human" + 0.005*"team" + 0.005*"earth" + 0.004*"guy" + 0.004*"alien" + 0.004*"fight" + 0.004*"special" + 0.004*"monster"
Topic: 4 
Words: 0.010*"famili" + 0.007*"father" + 0.007*"mother" + 0.006*"wife" + 0.006*"child" + 0.005*"relationship" + 0.005*"live" + 0.005*"husband" + 0.004*"emot" + 0.004*"daughter"
Topic: 5 
Words: 0.005*"murder" + 0.004*"mysteri" + 0.004*"style" + 0.004*"lead" + 0.004*"thriller" + 0.004*"viewer" + 0.004*"crime" + 0.003*"audienc" + 0.003*"shot" + 0.003*"element"
Topic: 6 
Words: 0.017*"seri" + 0.017*"book" + 0.015*"anim" + 0.012*"episod" + 0.012*"version" + 0.007*"lau

### 토픽 해석
- 토픽의 키워드들을 기반으로 아래와 같이 토픽의 이름을 유추할 수 있음<br>
  - Topic 1: Horror movies
  - Topic 2: Movies about families
  - Topic 3: SF Movies
  - Topic 4: Movies about families
  - Topic 5: Crime movies
  - Topic 6: Movies based on books
  - Topic 7: Generally bad movies 
  - Topic 8: Historical movies
  - Topic 9: Music movies
  - Topic 10: War movies

In [ ]:
# 토픽별 단어의 분포 확인: get_topics() 사용
topic_term_prob = lda_model.get_topics()   # vs get_topic_terms는 상위 10만
topic_term_prob  

array([[3.5015776e-07, 2.1638040e-04, 4.3846460e-04, ..., 3.4923252e-07,
        3.4919597e-07, 3.4929204e-07],
       [6.4221531e-05, 3.9156640e-04, 6.1038736e-04, ..., 2.5567062e-05,
        4.2081126e-07, 4.2076846e-07],
       [1.1954862e-04, 2.6300072e-04, 5.1240041e-04, ..., 2.1219596e-04,
        5.0889660e-04, 1.8055709e-04],
       ...,
       [4.3152829e-07, 2.4948348e-04, 1.5067502e-03, ..., 3.3525866e-06,
        4.3139343e-07, 4.3135248e-07],
       [2.6871123e-07, 5.1779969e-04, 1.9291530e-03, ..., 2.7111170e-07,
        3.9131476e-07, 4.9830805e-06],
       [1.0616987e-04, 2.2683846e-04, 1.2145453e-03, ..., 4.3234164e-07,
        4.0943840e-07, 4.0942695e-07]], dtype=float32)

In [ ]:
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10, 5000)
1.0


In [ ]:
# 문서별 토픽의 분포 확인: get_document_topics() 사용
sorted_doc_topic_list = []
for doc in corpus:  # 5만개의 문장에 대해서 ... 각문장 마다 ...
    doc_topic_dist = lda_model.get_document_topics(doc, minimum_probability=0)
    sorted_doc_topic = sorted (doc_topic_dist, key=lambda x:x[1], reverse=True)     # 확률로 소팅 x[1], 내림차순 reverse
    sorted_doc_topic_list.append (sorted_doc_topic)  # 누적

In [ ]:
# 문장마다 topic 확인
for i, doc in enumerate(sorted_doc_topic_list):
    if i >= 5: break
    print(doc)

[(4, 0.394905), (2, 0.24910812), (6, 0.119198635), (7, 0.08669371), (3, 0.063256115), (9, 0.046804816), (5, 0.036939755), (1, 0.0010312811), (8, 0.0010312736), (0, 0.0010312537)]
[(4, 0.54356545), (8, 0.28023818), (3, 0.16219209), (6, 0.0020010713), (9, 0.0020009154), (5, 0.002000587), (0, 0.0020005438), (2, 0.002000439), (7, 0.0020004348), (1, 0.0020003144)]
[(4, 0.22247426), (3, 0.2079549), (7, 0.18493468), (0, 0.15906505), (5, 0.14440322), (2, 0.07265487), (8, 0.0021283284), (6, 0.0021283117), (1, 0.002128284), (9, 0.0021280604)]
[(0, 0.51285285), (3, 0.34446386), (4, 0.12560618), (9, 0.0024398742), (6, 0.0024398018), (2, 0.0024396968), (1, 0.0024395972), (5, 0.002439472), (8, 0.0024393972), (7, 0.002439327)]
[(3, 0.33971903), (8, 0.2746204), (7, 0.23721251), (4, 0.13844553), (1, 0.0016672943), (5, 0.0016670953), (2, 0.0016670538), (6, 0.0016670531), (0, 0.001667045), (9, 0.0016669746)]


---

# <br>__3. Visualization__

In [ ]:
prepared_data = gensimvis.prepare(lda_model, corpus, dictionary) # 시각화준비 arg는 3가지

In [ ]:
# 각 토픽과 토픽의 키워드 시각화
# lambda를 조절하면 keyword score의 weight가 변경되어 새로운 키워드를 선택함
pyLDAvis.display(prepared_data)

# 5와 10의 주제는 다소 비슷하다? 고 볼수 있음.
# 결과인덱싱이 1~10 이다. python은 0 부터 인덱싱하므로 주의

In [ ]:
# save_html 로 저장한다.

---

# <br>__4. Finding Optimal Number of Topics__

In [ ]:
# num_topics_candidate에 속하는 다양한 토픽 개수를 기반으로 LDA 모델을 구축한 후, perplexity를 기준으로 최적 모델을 도출하는 함수
def find_optimal_model(corpus, dictionary, num_topics_candidate):
    best_perplexity = 0   # 로그취하므로 0 이 max임.
    best_model = None

    perplexity_list = []
    for num_topics in num_topics_candidate:
        model = LdaModel(corpus, id2word = dictionary, num_topics=num_topics, random_state=1234)
        perplexity = model.log_perplexity (corpus)
        perplexity_list.append (perplexity)
        
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_model = model
    
    best_num_topics = num_topics_candidate[np.argmin(perplexity_list)]   # np.min 은 최소값 리턴,  np.argmin은 최소값 인덱스(위치) 리턴
    print('Optimal number of topics: {}'.format(best_num_topics))  
    return best_model, perplexity_list

In [ ]:
# 시간 관계상 두 개의 값에 대하여 최적 토픽 개수 탐색
num_topics_candidate = [5, 10]
optimal_lda_model, perplexity_list = find_optimal_model (corpus, dictionary, num_topics_candidate)

Optimal number of topics: 10


In [ ]:
# 각 문서의 주요 토픽 도출 함수
def format_topics_sentences(lda_model, corpus, texts):
    # Init output
    sent_topics_df = []

    # 각 문서에서 주요 토픽 도출
    for i, row in enumerate(lda_model[corpus]):
        row = sorted(row, key=lambda x:x[1], reverse=True)   # key에 x[1] 확률값   이 큰것부터 소팅

        # 각 문서에 대하여 주요 토픽에 대한 확률 도출    --> 각 document, 즉, 행(row)에서 제일 처음 위치한 topic
        topic_num, prop_topic = row[0]  # => dominant topic  
        wp = lda_model.show_topic(topic_num)   # show_topic해서 dominent word 도출    --> list형태
        topic_keywords = ", ".join([word for word, prop in wp])   # string 형태로 형태변경,   word만 뽑아서 한번에 묶음.
        # wp안에는 word, prob가 있는데 이중에 word만 가져와서 .join 으로 list 형태로 만듬

        sent_topics_df.append([int(topic_num), round(prop_topic, 4), topic_keywords])   # dominent topic, 확률, 키워드 스트링 @ 한 문장당

    sent_topics_df = pd.DataFrame(sent_topics_df, columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # 결과 정리
    contents = pd.Series(texts)   # 데이터프레임에서 컬럼이 하나이므로 pd. Series로 만든다. 아래에서 concat 함
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)  # 원본데이터 옆에 붙임 (axis=1)
    sent_topics_df.reset_index(drop=True)  # reset_index 하고나면, 원래 index가 옆에 붙는데, 이를 drop True로 없앤다.
    sent_topics_df.columns = ['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']  # 'Text' 때문에 다시한번 .columns 로 네이밍

    return sent_topics_df

In [ ]:
# 각 문서의 주요 토픽 도출
df_dominant_topic = format_topics_sentences(optimal_lda_model, corpus, texts=documents['review'].values)
df_dominant_topic.head()

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,4,0.4105,"murder, mysteri, style, lead, thriller, viewer...",One of the other reviewers has mentioned that ...
1,4,0.5433,"murder, mysteri, style, lead, thriller, viewer...",A wonderful little production. <br /><br />The...
2,4,0.2224,"murder, mysteri, style, lead, thriller, viewer...",I thought this was a wonderful way to spend ti...
3,0,0.5128,"horror, killer, blood, hous, dead, zombi, gore...",Basically there's a family where a little boy ...
4,3,0.3396,"famili, father, mother, wife, child, relations...","Petter Mattei's ""Love in the Time of Money"" is..."


In [ ]:
# 각 토픽의 대표 문서 도출
#
# .groupby로 0~9로 그룹핑됨.  --> i 로 받음
# 

df_representative_doc = []
for i, grp in df_dominant_topic.groupby('Dominant_Topic'):   # for i , for grp 2번할꺼를 한번에 하네...
    df_representative_doc.append(grp.sort_values(['Topic_Perc_Contrib'], ascending=[0]).head(3).values.tolist()[0])   # top10은 head(10)

    # df_representative_doc.append(grp.sort_values(['Topic_Perc_Contrib'], ascending=[0]).head(1).values.tolist()[0])   # top10은 head(10)
    # grp 마다 확률이 높은순으로 소팅() .sort_values )  head(1)은 확률이 가장큰값 도출
    #  --> list로 만듬 (.values.tolist())

df_representative_doc = pd.DataFrame(df_representative_doc, columns=['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"])
df_representative_doc

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.9895,"horror, killer, blood, hous, dead, zombi, gore...","""House of the Damned"" (also known as ""Spectre""..."
1,1,0.9808,"school, kid, child, famili, home, wife, jack, ...",I taped The Morrison Murders on Lifetime Movie...
2,2,0.9780,"game, space, human, team, earth, guy, alien, f...",Getting Eaten By A Bunch Of Snakes Is More Ent...
3,3,0.9898,"famili, father, mother, wife, child, relations...","On paper, this movie would sound incredibly bo..."
4,4,0.9918,"murder, mysteri, style, lead, thriller, viewer...","After his earlier movie ""Videodrome"", which de..."
5,5,0.9866,"seri, book, anim, episod, version, laugh, read...","When I first heard about ""Down To Earth,"" I wa..."
6,6,0.9870,"worst, wast, bore, poor, terribl, review, aw, ...",simply i just watched this movie just because ...
7,7,0.9827,"american, documentari, countri, polit, french,...",This is a very funny Ealing comedy about a com...
8,8,0.9848,"beauti, excel, perfect, song, danc, episod, sc...","please, future writers, producers, directors -..."
9,9,0.9816,"black, fight, white, soldier, western, america...",For anyone who wishes to get an impression of ...


In [ ]:
# 각 문서의 주요 토픽을 기반으로 한 토픽별 문서 비율 도출
# 토픽별 문서의 개수 도출
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts()
topic_counts = topic_counts.to_frame().sort_index()   # sort_index() 하면 index기준으로 소팅

# 토픽별 문서 개수의 비율 도출
topic_contribution = df_dominant_topic['Dominant_Topic'].value_counts(normalize=True)
topic_contribution = topic_contribution.to_frame().sort_index()

# 토픽별 키워드 도출
topic_num_keywords = df_representative_doc[['Topic_Num', 'Keywords']]

# 결과 정리
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,0,"horror, killer, blood, hous, dead, zombi, gore...",4446,0.08892
1,1,"school, kid, child, famili, home, wife, jack, ...",3099,0.06198
2,2,"game, space, human, team, earth, guy, alien, f...",2881,0.05762
3,3,"famili, father, mother, wife, child, relations...",5499,0.10998
4,4,"murder, mysteri, style, lead, thriller, viewer...",5406,0.10812
5,5,"seri, book, anim, episod, version, laugh, read...",4478,0.08956
6,6,"worst, wast, bore, poor, terribl, review, aw, ...",11967,0.23934
7,7,"american, documentari, countri, polit, french,...",2640,0.05280
8,8,"beauti, excel, perfect, song, danc, episod, sc...",7068,0.14136
9,9,"black, fight, white, soldier, western, america...",2516,0.05032


---